## Data Import

In [120]:
# Import basic packages lik numpy, pandas, math plot and seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.svm import SVC

In [105]:
# import dataset
df = pd.read_csv('https://raw.githubusercontent.com/lijjumathew/MSDS-Machine-Learning-1-Project/master/dataset/Telco-Customer-Churn.csv')

## Clean Up Data and Create Dummy Variables

In [106]:
#Ideally SeniorCitizen column should be a factor, so let's convert 1,0 values to Yes,No and later we can label encode all factor columns
df.SeniorCitizen=df.SeniorCitizen.apply(lambda x: 'Yes' if x==1 else 'No')

# Getting rid of unwanted columns like Customer Id.
if 'customerID' in df:
    del df['customerID']
    
# converting TotalCharges object dataset into numeric
# errors = 'coerce’ means, if invalid parsing occur then set NaN
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors = 'coerce')

# Replacing blank values with nulls.
df=df.replace(r'^\s*$', np.nan, regex=True)

# Total charges has some blank values/missing values and needs to be imputed. Filling the missing values
df["TotalCharges"].fillna(df["TotalCharges"].mean(), inplace=True)

# Consolidate MultipleLines attribute
df['MultipleLines'] = df['MultipleLines'].replace('No phone service','No')

# Change all values of 'No internet service' to 'No'
df = df.replace('No internet service','No')

# Replace all yes/no values with 1/0
df = df.replace(to_replace=['Yes','No'], value=[1,0])

# Create dummy variables in the entire dataset
df = pd.get_dummies(df)

## Logistic Regression

In [112]:
# Separate the churn results from the dataset
y = df['Churn'].values
x = df.drop(columns = ['Churn'])

# Set features
features = x.columns.values

# Normalize values
scale = MinMaxScaler(feature_range = (0,1))
scale.fit(x)
x = pd.DataFrame(scale.transform(x))
x.columns = features

In [142]:
# Set up train/test split with 80/20 ratio
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 99)

# Create the model and fit
model = LogisticRegression()
fit = model.fit(x_train,y_train)
predict = model.predict(x_test)
print("Our accuracy score of the model using logistic regression is: ",metrics.accuracy_score(y_test,predict))
print(classification_report(y_test,predict))

0.6666666666666666
0.538860103626943
Our accuracy score of the model using logistic regression is:  0.7998580553584103
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1023
           1       0.67      0.54      0.60       386

    accuracy                           0.80      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.79      0.80      0.79      1409



In [129]:
# Calculate and display weights from logistic regression
weights = pd.Series(model.coef_[0], index=x.columns.values)
print(weights.sort_values(ascending = False))

TotalCharges                               0.987159
InternetService_Fiber optic                0.822797
Contract_Month-to-month                    0.698867
PaperlessBilling                           0.371334
PaymentMethod_Electronic check             0.292634
MultipleLines                              0.240501
SeniorCitizen                              0.233807
StreamingTV                                0.181166
StreamingMovies                            0.180732
gender_Female                              0.016114
InternetService_DSL                        0.014568
Contract_One year                          0.007922
MonthlyCharges                             0.000814
gender_Male                               -0.015305
PaymentMethod_Mailed check                -0.020221
DeviceProtection                          -0.023484
Partner                                   -0.056451
PaymentMethod_Credit card (automatic)     -0.118700
Dependents                                -0.131897
OnlineBackup

Immediate thoughts on the weights, TotalCharges makes sense. Seeing the month-to-month up there is not surprising and supports our initial EDA findings. Tenure being the least likely to churn also lines up with our EDA findings, most people were likely to churn within the first few months.

## Support Vectors

In [141]:
# Fit and predict using SVM
# Support Vectors are the data points that are closest to the edges of the margin of the separating line.
# The separating line and margins are called the hyperplane
svm = SVC(kernel='rbf',cache_size=1000)
svm.fit(x_train,y_train)
predict = svm.predict(x_test)
print("Our accuracy score of the model using SVM is: ",metrics.accuracy_score(y_test,predict))
print("This is about the same as our logistic regression model")

0.6725352112676056
0.4948186528497409
Our accuracy score of the model using SVM is:  0.7955997161107168
This is about the same as our logistic regression model


In [123]:
# Confusion matrix
print(confusion_matrix(y_test,predict))

[[916 107]
 [189 197]]
